#### Date - reading the csv

In [8]:
date_csv = pd.read_csv('./data_sql/date.csv', sep=';')

In [12]:
date_csv['day_number'] = date_csv['day_id'].copy()
date_csv['month_number'] = date_csv['month_id'].copy()

In [23]:
def get_month_name(month_id):
    month_row = month[month['month_number']==month_id]
    return month_row['month_name'].iloc[0]

In [29]:
size_date = len(date_csv.index)
date_csv['month_name'] = [i for i in range(size_date)]

for i in date_csv['month_id'].index:
    month_name = get_month_name(date_csv['month_id'][i])
    date_csv['month_name'][i] = month_name

C:\Users\filip\AppData\Local\Temp\ipykernel_3684\4036878054.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_csv['month_name'][i] = month_name


In [34]:
def get_year_number(year_id):
    return 2010 + year_id -1

In [39]:
size_date = len(date_csv.index)
date_csv['year_number'] = [i for i in range(size_date)]

for i in date_csv['year_id'].index:
    year_number = get_year_number(date_csv['year_id'][i])
    date_csv['year_number'][i] = year_number

C:\Users\filip\AppData\Local\Temp\ipykernel_3684\1921800919.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_csv['year_number'][i] = year_number


In [41]:
date_csv.to_csv('./data_sql/date.csv', sep=";", index=False)

## Location

* Cascata

In [44]:
location = pd.DataFrame()
location['latitude'] = listings['latitude']
location['longitude'] = listings['longitude']

In [45]:
location = location.drop_duplicates().reset_index(drop=True)

In [58]:
state = pd.DataFrame({'state_id':1, 'state':['Porto'], 'country_id': 1})

In [62]:
country = pd.DataFrame({'country_id': 1, 'country': ['Portugal'], 'country_code': ['PT']})

# Create a new dataframe that came from listings that take the city

city = pd.DataFrame()


city["city"] = listings["neighbourhood_group_cleansed"].drop_duplicates().reset_index(drop=True)
size_city = len(city.index)
city["city_id"] = [i for i in range(1, size_city+1)]
city["country_id"] = 1

# Create a new dataframe that came from listings that take the neighbourhood

neighbourhood = pd.DataFrame()

# Extract the 'neighbourhood_cleansed' column and drop duplicates
neighbourhood['combined_neighbourhood'] = listings['neighbourhood_cleansed'].drop_duplicates().reset_index(drop=True)

# Assuming 'city_column' is the column in 'listings' containing city information
neighbourhood = pd.merge(neighbourhood, listings[['neighbourhood_cleansed', 'neighbourhood_group_cleansed']], 
                         left_on='combined_neighbourhood', right_on='neighbourhood_cleansed', how='left')

neighbourhood.rename(columns={'neighbourhood_group_cleansed': 'city_column'}, inplace=True)
neighbourhood.drop(['neighbourhood_cleansed'], axis=1, inplace=True)
neighbourhood.drop_duplicates().reset_index(drop=True, inplace=True)


# Split the combined neighborhood string into individual neighborhoods using both commas and " e "
neighbourhood['neighbourhood'] = neighbourhood['combined_neighbourhood'].str.split(',| e ')

# Explode the DataFrame, creating a new row for each neighborhood
neighbourhood = neighbourhood.explode('neighbourhood')

# Drop unnecessary columns and rename columns
neighbourhood.rename(columns={'city_column': 'city'}, inplace=True)
neighbourhood.drop_duplicates().reset_index(drop=True, inplace=True)
neighbourhood = pd.merge(neighbourhood, city, on="city", how='left')
neighbourhood.drop(['combined_neighbourhood', 'city', 'country_id'], axis=1, inplace=True)

In [63]:
city.rename(columns={'country_id':'state_id'}, inplace=True)

In [67]:
city.drop(['city_id'], axis=1, inplace=True)

In [78]:
neighbourhood = neighbourhood.drop_duplicates().reset_index(drop=True)

In [92]:
neighbourhood

,neighbourhood,city_id
0,Vila Chã,1
1,Lordelo do Ouro,2
2,Massarelos,2
3,Cete,3
4,Cedofeita,2
...,...,...
227,Guilhabreu,1
228,Arrifana,11
229,Vilela,3
230,Vilarinho,16


In [142]:
def get_neighbourhood_id(listing_id):
    listing_location = listings[listings['id']==listing_id]
    list_neighbourhood = listing_location['neighbourhood'].iloc[0]

    is_neighbourhood_nan = pd.isna(list_neighbourhood)
    if is_neighbourhood_nan:
        return pd.NA
    
    print(list_neighbourhood)
    

In [143]:
get_neighbourhood_id(41339)

Porto, Portugal


## Host

In [12]:
# selected_columns = ['host_response_time', 'host_response_rate', 'host_acceptance_rate']
selected_columns = ['host_id', 'host_name', 'host_url', 'host_total_listings_count', 'host_has_profile_pic', 'host_identity_verified', 'host_is_superhost', 'host_about',
                    'host_response_time', 'host_response_rate', 'host_acceptance_rate', 
                    'host_verifications']

host = listings[selected_columns].copy()
host.rename(columns={'host_about':'description'}, inplace=True)

In [13]:
selected_columns = ['host_response_time', 'host_response_rate', 'host_acceptance_rate']

host_response = listings[selected_columns].copy()

In [14]:
host_response = host_response.drop_duplicates().reset_index(drop=True)

In [20]:
host_response = host_response.fillna(-1)

In [22]:
# TODO: Compare values NaN

def get_host_response_id(host_id):
    listing_host = listings[listings['host_id']==host_id]
    list_host_response_time = listing_host['host_response_time'].iloc[0]
    list_host_response_rate = listing_host['host_response_rate'].iloc[0]
    list_host_acceptance_rate = listing_host['host_acceptance_rate'].iloc[0]

    # Check if values are NaN
    is_response_time_nan = pd.isna(list_host_response_time)
    is_response_rate_nan = pd.isna(list_host_response_rate)
    is_acceptance_rate_nan = pd.isna(list_host_acceptance_rate)

    if is_response_time_nan:
        list_host_response_time = -1
    if is_response_rate_nan:
        list_host_response_rate = -1
    if is_acceptance_rate_nan:
        list_host_acceptance_rate = -1

    row = host_response[(host_response['host_response_time']==list_host_response_time)
                         & (host_response['host_response_rate']==list_host_response_rate)
                         & (host_response['host_acceptance_rate']==list_host_acceptance_rate)]
    
    return row.index[0] + 1 

In [25]:
# host_response_id
size_host = len(host.index)
host['host_response_id'] = [i for i in range(size_host)]

for i in host['host_id'].index:
    host_id = host['host_id'][i]
    host['host_response_id'][i] = get_host_response_id(host_id)

C:\Users\filip\AppData\Local\Temp\ipykernel_4768\692988943.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  host['host_response_id'][i] = get_host_response_id(host_id)


In [29]:
host_verification = pd.DataFrame()
host_verification['host_verifications'] = listings['host_verifications']

In [30]:
host_verification = host_verification.drop_duplicates().reset_index(drop=True)

In [32]:
def get_host_verifications_id(host_id):
    listing_host = listings[listings['host_id']==host_id]
    list_host_verifications = listing_host['host_verifications'].iloc[0]
    
    row = host_verification[host_verification['host_verifications']==list_host_verifications]
    return row.index[0] + 1 

In [33]:
# host_verifications_id -> run
size_host = len(host.index)
host['host_verifications_id'] = [i for i in range(size_host)]

for i in range(size_host):
    host_id = host['host_id'][i]
    host['host_verifications_id'][i] = get_host_verifications_id(host_id)

C:\Users\filip\AppData\Local\Temp\ipykernel_4768\482111441.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  host['host_verifications_id'][i] = get_host_verifications_id(host_id)


In [37]:
host.columns

Index(['host_id', 'host_name', 'host_url', 'host_total_listings_count',
       'host_has_profile_pic', 'host_identity_verified', 'host_is_superhost',
       'description', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_verifications', 'host_response_id',
       'host_verifications_id'],
      dtype='object')

In [38]:
host = host.drop_duplicates().reset_index(drop=True)

In [39]:
host.to_csv('./data_sql/host.csv', sep=";", index=False)

## Property

In [4]:
selected_columns = ['accommodates', 'bedrooms', 'beds', 'amenities', 'room_type', 'property_type', 'bathrooms_text']

property = listings[selected_columns].copy()
property.rename(columns={'bathrooms_text':'bathrooms'}, inplace=True)

In [40]:
property['property_id'] = listings['id']

In [6]:
property = property.drop_duplicates().reset_index(drop=True)

In [42]:
property.columns

Index(['accommodates', 'bedrooms', 'beds', 'amenities', 'room_type',
       'property_type', 'bathrooms', 'property_id'],
      dtype='object')

In [43]:
property.to_csv('./data_sql/property.csv', sep=";", index=False)

# Review

In [183]:
review = pd.DataFrame()
review['reviewer_id'] = reviews['reviewer_id']
review['review'] = reviews['comments']


In [82]:
def get_date_id(date_str):
    return date[date['date']==date_str].index[0]+1

In [6]:
# data_id
size_review = len(review.index)
review['date_id'] = [i for i in range(size_review)]

for i in range(size_review):
    review['date_id'][i] = get_date_id(reviews['date'][i])


KeyboardInterrupt



In [214]:
def get_host_id(listings_id):
    l = listings[listings['id']==listings_id]
    return l['host_id'].tolist()[0]

In [212]:
# host_id -> review['listing_id'] = listings['id]->host_id

size_review = len(review.index)
review['host_id'] = [i for i in range(size_review)]

for i in range(size_review):
    review['host_id'][i] = get_host_id(reviews['listing_id'][i])

# Listing

In [44]:
#listing -> listing_id = id, host_id = host_id, price -> price

# selected_columns = ['availability_30', 'availability_60', 'availability_90', 'availability_365']
# selected_columns = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']

selected_columns = ['id', 'host_id', 'price', 'description', 
                    'availability_30', 'availability_60', 'availability_90', 'availability_365', 
                    'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']
listing = listings[selected_columns].copy()
listing.rename(columns={'id':'listing_id'}, inplace=True)